## Homework

Search using elastic search

### Question 1: Running elastic

#Running on Elastic search 8.17.6
```bash
$ curl localhost:9200
{
  "name" : "906ced9add0e",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "cuMnxk0cTJaRVufQwW9Paw",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}
```

"build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",

### Question 2: Indexing the data

In [12]:
#importing the libraries
import json
import requests
from google import genai
from elasticsearch import Elasticsearch
from tqdm import tqdm

In [6]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [9]:
# create es client
es_client = Elasticsearch('http://localhost:9200')

In [10]:
#run the index settings
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "dynamic": "strict",  # Prevent unwanted field additions
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

#give the index a name
index_name = "course-questions"

# Delete old index if exists
#if es_client.indices.exists(index=index_name):
#    es_client.indices.delete(index=index_name)
    
#create the index
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [15]:
#iterate through the document to add it to ES
# this failed initially becuase I wa passing the entire docs instead of doc (single line at a time)
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:04<00:00, 227.44it/s]


### Question 3: Searching

In [43]:
query = "How do execute a command on a Kubernetes pod?"

In [44]:
search_query= {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }
    
response = es_client.search(index=index_name, body=search_query)
print(response)

result_doc = []
for hit in response['hits']['hits']:
    result_doc.append(hit['_source'])

print(result_doc)

{'took': 11, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': 'Oa66c5cBhaU4gpfM9myO', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'yK66c5cBhaU4gpfM-Gya', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashboard', '

### Question 4: Filtering

In [46]:
query = "How do copy a file to a Docker container?"

In [47]:
search_query= {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
response = es_client.search(index=index_name, body=search_query)
print(response)

result_doc = []
for hit in response['hits']['hits']:
    result_doc.append(hit['_source'])

print(result_doc)

{'took': 11, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 340, 'relation': 'eq'}, 'max_score': 73.38676, 'hits': [{'_index': 'course-questions', '_id': 'Oa66c5cBhaU4gpfM9myO', '_score': 73.38676, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'WK66c5cBhaU4gpfM9mz_', '_score': 66.688705, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or dire

### Question 5: Building a prompt

In [35]:
context_template  = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

context = ""

for doc in result_doc:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = context_template.format(question=query, context=context).strip()

len(prompt)
#print(prompt)

1621

### Question 6: Tokens

In [38]:
import tiktoken

In [39]:
encoding = tiktoken.encoding_for_model("gpt-4o")
encoding

<Encoding 'o200k_base'>

In [41]:
num_tokens = len(encoding.encode(prompt))
num_tokens

354

### Bonus responses: Generating answers

In [58]:
gemini_key = "AIzaSyCWYdzvCfj_...-v5z5A_1CkE7vc"

In [49]:
def llm_response(prompt):
    client = genai.Client(api_key=gemini_key)
    response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
    )

    return print(response.text)

In [51]:
answer = llm_response(prompt)
answer

You can copy files from your local machine into a Docker container using the `docker cp` command.
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows: `docker cp /path/to/local/file_or_directory container_id:/path/in/container`.

In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows: `COPY ["src/predict.py", "models/xgb_model.bin", "./"]`.



### Bonus responses: calculating the costs (ungraded)

Considering that I am using Gemini 2.5 Flash Preview. Here are the pricing as at June 15th, 2025 per 1Million token
- Input: $0.15
  
- Output: Non-thinking: $0.60 | Thinking: $3.50

In [57]:
client = genai.Client(api_key=gemini_key)
answer = client.models.generate_content(
model="gemini-2.0-flash",
contents=prompt
)
answer

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='You can copy files from your local machine into a Docker container using the `docker cp` command. The basic syntax is as follows: `docker cp /path/to/local/file_or_directory container_id:/path/in/container`.\n\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over with the syntax: `COPY ["src/predict.py", "models/xgb_model.bin", "./"]`.\n')], role='model'), citation_metadata=CitationMetadata(citations=[Citation(end_index=204, license=None, publication_date=None, start_index=73, title=None, uri='https://docs.google.com/document/d/1LpPanc33QJJ6BSsyxVg-pWNMplal84TdZtq10naIhD8/edit')]), finish_message=None, token_count=None, finish_reason=<FinishReason.STOP: 'STOP'>, url_context_metadata=

**Current usage based on the API output above:**
- Input tokens: 387 (prompt_token_count)
- Output tokens: 99 (candidates_token_count)
- Total tokens: 486 (total_token_count)

**Pricing for Gemini:**
- Input: $0.15 per 1M tokens
- Output: $0.60 per 1M tokens (standard request)

**Cost Calculation:**
- Input cost: (387 / 1,000,000) * $0.15 = $0.00005805
- Output cost: (99 / 1,000,000) * $0.60 = $0.0000594
- Total cost: $0.00005805 + $0.0000594 = $0.00011745 (~0.0117 cents

**For 1000 requests, we would be spending: $0.00011745x1000 = $0.11745